In [1]:
import datetime
import time

import pandas as pd
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pgpasslib

from querymsc import WorkflowTools

In [2]:
connection_string = "postgresql+psycopg2://nannau@dbmsc.pcic.uvic.ca/msc"
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
start_time = datetime.datetime(1990, 1, 1)
end_time = datetime.datetime(1991, 1, 1)

msc = WorkflowTools(start_time, end_time, month=1)

In [4]:
annual_tot_precip = msc.query_annual_precip(session)
annual_tot_rain = msc.query_annual_rain(session)

Year interval: 1.0


In [5]:
query = msc.query_design_temp_percentile(session)

SELECT percentile_cont(%(percentile_cont_1)s) WITHIN GROUP (ORDER BY crmp.obs_raw.datum ASC) * %(param_1)s AS temp, min(crmp.obs_raw.obs_time) AS time_min, max(crmp.obs_raw.obs_time) AS time_max, crmp.meta_history.lat AS lat, crmp.meta_history.lon AS lon, crmp.meta_history.station_id AS station_id, count(crmp.obs_raw.datum) / %(count_1)s AS completeness 
FROM crmp.obs_raw JOIN crmp.meta_vars ON crmp.obs_raw.vars_id = crmp.meta_vars.vars_id JOIN crmp.meta_history ON crmp.obs_raw.history_id = crmp.meta_history.history_id 
WHERE crmp.obs_raw.obs_time >= %(obs_time_1)s AND crmp.obs_raw.obs_time < %(obs_time_2)s AND EXTRACT(month FROM crmp.obs_raw.obs_time) = %(param_2)s AND crmp.meta_vars.net_var_name = %(net_var_name_1)s AND crmp.meta_vars.standard_name = %(standard_name_1)s AND crmp.meta_vars.cell_method = %(cell_method_1)s AND crmp.obs_raw.datum != %(datum_1)s GROUP BY crmp.meta_history.lat, crmp.meta_history.lon, crmp.meta_history.station_id


In [10]:
pd.read_sql(annual_tot_precip.statement, engine).head()

,annual_precip,min_date,max_date,completeness,lat,lon,station_id
0,1052.0,1990-01-01,1990-12-31,1.0,41.7833,-82.6833,6063
1,1159.7,1990-01-01,1990-12-31,1.0,41.9500,-82.5167,6140
2,1061.4,1990-01-01,1990-12-31,1.0,42.0333,-82.9000,6036
3,1144.4,1990-01-01,1990-12-31,1.0,42.0431,-82.6739,6040
4,1189.5,1990-01-01,1990-12-31,1.0,42.1034,-83.0945,5992


In [11]:
pd.read_sql(annual_tot_rain.statement, engine).head()

,annual_rain,min_date,max_date,completeness,lat,lon,station_id
0,1013.6,1990-01-01,1990-12-31,1.0,41.7833,-82.6833,6063
1,1102.7,1990-01-01,1990-12-31,1.0,41.9500,-82.5167,6140
2,990.4,1990-01-01,1990-12-31,1.0,42.0333,-82.9000,6036
3,1094.2,1990-01-01,1990-12-31,1.0,42.0431,-82.6739,6040
4,1097.7,1990-01-01,1990-12-31,1.0,42.1034,-83.0945,5992


In [8]:
df_vars = pd.read_sql(msc.query_variables(session).statement, engine)
#df_vars[df_vars['net_var_name']=='127']
#df_vars[(df_vars['net_var_name'].astype(int) > 124) & (df_vars['net_var_name'].astype(int) < 132)]
df_vars[df_vars['standard_name'].str.contains('lwe')]

,vars_id,standard_name,long_description,unit,cell_method,net_var_name
72,1361,lwe_thickness_of_precipitation_amount,precipitation amount (minutes 00-60),0.1 mm,time: sum,262
73,1362,lwe_thickness_of_precipitation_amount,precipitation amount (minutes 00-15),0.1 mm,time: sum,263
74,1363,lwe_thickness_of_precipitation_amount,precipitation amount (minutes 15-30),0.1 mm,time: sum,264
75,1364,lwe_thickness_of_precipitation_amount,precipitation amount (minutes 30-45),0.1 mm,time: sum,265
76,1365,lwe_thickness_of_precipitation_amount,precipitation amount (minutes 45-60),0.1 mm,time: sum,266
108,1397,lwe_thickness_of_precipitation_amount,Total precipitation,0.1 mm,time: sum,12
119,1412,lwe_thickness_of_precipitation_amount,1200 UTC,0.1 mm,time: sum,6
120,1413,lwe_thickness_of_precipitation_amount,1800 UTC,0.1 mm,time: sum,7
121,1414,lwe_thickness_of_precipitation_amount,0000 UTC,0.1 mm,time: sum,8
122,1415,lwe_thickness_of_precipitation_amount,0600 UTC,0.1 mm,time: sum,9
